# Chapo_tfidf

## Définition des fonctions

In [11]:
import os
import json

from nltk import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import math

# Fonction pour nettoyer le texte des caractères spéciaux
def clean_text(text, characters):
    c_text = text
    for char in characters:
        c_text = c_text.replace(char, " ")
    c_text = c_text.lower()
    return(c_text)

# Calcul de document frequency
def df(data):
    df = {}
    for desc in data.values():
        for token in set(word_tokenize(desc)):
            stem = stemmer.stem(token)
            if stem in df:
                df[stem] += 1
            else:
                df[stem] = 1
    return df

In [12]:
# Récupération des mots stemmés
def stem_words(data):
    stem_dict = {}
    for k, desc in data.items():
        tokens = word_tokenize(desc)
        for t in tokens:
            token_stem = stemmer.stem(t)
            if token_stem in stem_dict:
                stem_dict[token_stem].append(t)
            else:
                stem_dict[token_stem] = [t]

    for k, v in stem_dict.items():
        stem_dict[k] = list(set(stem_dict[k]))
    
    return stem_dict

In [13]:
# Tri des mots par leur valeur de tf-idf.
def sorted_tfidf(tf_idf, stem_dict, nb_mots=15):
    sorted_tfidf = {}
    for clus, dico in tf_idf.items():
        list_temp = [(j, i) for i, j in dico.items()]  # on met la valeur du tfidf du mot en 1er attribut
        list_temp = sorted(list_temp, reverse=True)  # on ordonne la liste suivant les valeurs de tfidf décroissante
        tfidf_clus = [(j,i) for i, j in list_temp]
        sorted_clus = []
        for (token, j) in tfidf_clus:
            token = stem_dict[token]
            sorted_clus.append([token, j])
        sorted_tfidf[clus] = sorted_clus[0:nb_mots]
        
    final_tfidf = {}
    for k, v in sorted_tfidf.items():
        final_tfidf[k] = []
        for item in v:
            final_tfidf[k].append(item[0][0])
    return final_tfidf

# tf-idf for all texts
def tf_idf(data, nb_mots):
    nb_texts = len(data.keys())
    term_freq = df(data)

    tf_idf ={}
    for k, desc in data.items():
        tf_idf_clus = {}
        tokens = word_tokenize(desc)
        tokens = [stemmer.stem(t) for t in tokens]
        frequencies = FreqDist(tokens)

        for token, freq in frequencies.items():
            tf_idf_clus[token] = freq * math.log( nb_texts / (1 + term_freq.get(token, 0)) )

        tf_idf[k] = tf_idf_clus

    stem_dict = stem_words(data)
    return sorted_tfidf(tf_idf,stem_dict,nb_mots)

## Application

### Préparation des chemins

In [14]:
dir_path = os.getcwd()
data_dir_path = os.path.join(dir_path, '00 - Scraping output')
jobs_list = os.listdir(data_dir_path)

### Options

In [15]:
# Stemmer
stemmer = SnowballStemmer('french')
# Choix du nombre de mots pour le texte
nb_mots = 30
# Charactères à retirer
special_chars = [",", "\n",".", "'", ";", "\\n",'"',")","(","»","«", '?','0','1','2','3','4','5','6','7','8','9','—','+','-']

### Chargement des données et calculs

In [20]:
# tf-idf sur les chapo des métiers
data = {}
for job in jobs_list:
    metier_dir_path = os.path.join(data_dir_path, job)
    with open(metier_dir_path, 'r') as f:
        metier = json.load(f)
    data[metier["Métier"]] = metier["chapo"]
    for item in metier["Le métier"]:
        data[metier["Métier"]] = data[metier["Métier"]] + ' ' + item

In [22]:
data["pilote de ligne"]

"Aujourd'hui Tokyo, demain New York, Dakar la semaine prochaine... Le métier de pilote de ligne fascine. Mais ce professionnel du ciel doit garder les pieds sur terre : il est responsable de la vie de centaines de passagers. Le métier Nature du travail  Préparer le plan de vol Avant le départ, le pilote établit minutieusement le plan de vol en tenant compte de la météorologie, du trajet et de ses particularités ou difficultés... En fonction du poids de l'avion et de la destination, il calcule la quantité de carburant minimum à emporter. Aidé du copilote, il prépare les instruments de navigation et entre les données dans l'ordinateur de vol. Quand tout est prêt, place à l'ultime récapitulation, à voix haute, des consignes de sécurité. Le commandant de bord demande alors l'autorisation de décoller à la tour de contrôle.  Piloter et garder le contact Tout au long du vol, dans le cockpit, le pilote surveille l'ensemble des paramètres sur le tableau de bord (direction, altitude, météo, cons

In [23]:
# Nettoyage de la ponctuation dans les définitions des métiers
clean_data = {}
for metier, text in data.items():
    text_str = str(text)
    text_c = clean_text(text_str, special_chars)
    clean_data[metier] = text_c
data2 = clean_data

# TF-IDF et sélection des meilleurs résultats
res_tf_idf = tf_idf(data2, nb_mots)

### Résultats

In [52]:
# Afficher les résultats de tous les texts
metier

{"Centre(s) d'intérêt ": ["Enquêter - analyser l'information - je veux en faire mon travail",
  "J'ai le sens du contact",
  "J'aimerais informer - communiquer"],
 'Métier': 'bibliothécaire',
 "Niveau d'accès ": ['bac + 2'],
 'Salaire débutant  ': ['1700 euros'],
 'Secteur(s) professionnel(s) ': ["Culture - Patrimoine - Artisanat d'art",
  'Enseignement - Formations',
  'Fonction publique',
  'Sport - Loisirs - Tourisme'],
 'Statut(s) ': ['Statut fonctionnaire', 'Statut salarié'],
 'Synonymes ': ['Conservateur/trice ', 'Médiathécaire-documentaliste '],
 'chapo': "Dans une bibliothèque municipale ou à la Bibliothèque nationale de France, le bibliothécaire est le trait d'union entre les ouvrages et les usagers. Accueil du public, conservation du fonds, classement, gestion du prêt... ses tâches sont nombreuses."}

In [13]:
# Afficher les résultats d'un text en particulier
key = "A1101"
print(tf_idf[key])

['forestière', 'agricoles', 'engins', 'coopérative', 'r']


In [24]:
# Copie du dictionnaire au format JSON
output_dir_path = os.path.join(dir_path, 'chapo_lemetier_tfidf.json')
with open(output_dir_path, 'w') as f:
    json.dump(res_tf_idf, f, indent=4)